## 0.0 파이썬 모듈 불러들이기

In [7]:
import numpy as np
import csv
# import time

np.random.seed(1234)
# def randomize(): np.random.seed(time.time())

## 0.1 하이퍼 파라미터 정의

In [8]:
# 평균
RND_MEAN = 0

# 표준편차
RND_STD = 0.0030

# 학습률
LEARNING_RATE = 0.001

## A.1 실험용 메인함수

In [9]:
def abalone_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_abalone_dataset()      # 데이터를 불러들이는 함수
    init_model()                # 모델 초기화 함수
    train_and_test(epoch_count, mb_size, report)    # 학습 및 테스트 수행 함수

## B.1 데이터 적재함수 정의 

In [10]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        # csv 파일의 데어터를 반복문을 활용하여 rows 라는 빈 리스트에 저장
        for row in csvreader:
            rows.append(row)
            
    # 이후에 다른 변수에서 활용하고자 전역변수 생성
    global data, input_cnt, output_cnt

    # 데이터의 입출력 벡터 정보를 저장, 이후 크기 지정에 활용
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    # 원-핫 벡터 처리
    # I = 1,0,0 / M = 0,1,0 / F = 0,0,1
    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

## B.2 파라미터 초기화 함수 정의 

In [11]:
def init_model():
    # 전역변수 불러오기 및 생성
    global weight, bias, input_cnt, output_cnt

    # 가중치 초기화
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    
    # 편향 초기화
    bias = np.zeros([output_cnt])

- np.random.normal(평균, 표준편차, 크기) : 정규분포를 갖는 난수 생성

- np.zeros(shape) : 0 으로 가득찬 array를 생성

## B.3 학습 및 평가 함수 정의

In [12]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    # epoch_count 만큼 '에폭' 반복 수행
    # 한 차례의 에폭마다의 손실과 정확도 저장
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        # 학습데이터 크기에 비례하여(80%) 미니배치 처리된 횟수만큼 반복 수행
        for n in range(step_count):
            # 미니배치 마다의 학습 데이터 분할
            train_x, train_y = get_train_data(mb_size, n)

            # 학습 수행 및 손실과 정확도 산출
            loss, acc = run_train(train_x, train_y)

            # 미니배치 처리 이후 손실과 정확도를 누적하여 저장(이후 이 값들을 평균내면 한 차례의 '에폭' 처리)
            losses.append(loss)
            accs.append(acc)
            
        # 출력 주기 및 테스트 주기 설정
        if report > 0 and (epoch+1) % report == 0:
            # 테스트 데이터로 테스트 진행
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))

    # 모든 반복이 종료되었을때, 한번 더 최종 결과 출력        
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

## C.1~3 학습 및 평가 데이터 획득 함수 정의

In [13]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx

    # 데이터의 순서값을 생성
    shuffle_map = np.arange(data.shape[0])

    # 데이터를 무작위로 섞어주는 과정
    np.random.shuffle(shuffle_map)

    # 데이터의 80% 기준, 미니배치 사이즈에 의한 1 에폭당 미니배치 횟수 출력
    step_count = int(data.shape[0] * 0.8) // mb_size

    # 학습 데이터와 테스트 데이터의 경계선 인덱스 생성
    test_begin_idx = step_count * mb_size

    return step_count


def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt

    # 테스트 데이터 생성
    test_data = data[shuffle_map[test_begin_idx:]]

    # 테스트 데이터의 종속변수, 독립변수 분할
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]


def get_train_data(mb_size, nth):
    # mb_size: 미니배치 크기, nth: 미니배치 실행 순서
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        # 첫 에폭마다 한하여
        # 처음부터 경계선까지 인덱스를 섞어준다.
        np.random.shuffle(shuffle_map[:test_begin_idx])

        # 섞인 인덱스로 미니배치 크기에 맞게 데이터 분할 및 train_data로 저장
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

## C.4~5 학습 실행 함수와 평가 실행함수 정의

In [14]:
def run_train(x, y):
    '''순전파 및 정확도 추출 과정'''

    # 신경망 연산 부분(보조정보: 입력 벡터 x)
    output, aux_nn = forward_neuralnet(x)

    # 신경망 후처리 과정(손실함수 구하는 과정)(보조정보: 편차 diff)
    loss, aux_pp = forward_postproc(output, y)

    # 정확도를 구하는 과정
    accuracy = eval_accuracy(output, y)
    
    '''역전파 과정'''

    # 항상 순전파의 역순으로 수행
    # 순전파 때 출력이었던 성분의 '손실함수의 기울기'
    G_loss = 1.0

    # 손실함수의 처리과정인 '평균제곱오차'의 역순, 즉 'MSE의 역전파 처리'
    G_output = backprop_postproc(G_loss, aux_pp)

    # 입력값에 따른 f(x)에 대한 편미분 과정을 구해주는 내부처리
    # 직접적인 학습이 이뤄지는 부분(가중치와 편향이 학습률을 활용하여 실제 학습 과정 수행)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy


def run_test(x, y):
    # 순전파 과정 수행(두번째 반환값인 '추가정보 바환'은 필요 없으므로 "_"처리)
    output, _ = forward_neuralnet(x)

    # 최종 정확도 추출
    accuracy = eval_accuracy(output, y)

    return accuracy

## D.1 / E.1 단층 퍼셉트론에 대한 순전파 및 역전파 함수 정의

In [15]:
def forward_neuralnet(x):
    # 전역변수 셋팅
    global weight, bias

    # 편향이 더해진 입력 벡터와 가중치 벡터에 대한 기본적인 신경망 연산식
    output = np.matmul(x, weight) + bias

    # 역전파에 필요한 보조정보로 활용
    return output, x

# 입력값에 따른 f(x)에 대한 편미분 과정에서 각각 가중치(G_W)와 편향(G_B)의 손실 기울기 연산
def backprop_neuralnet(G_output, x):
    global weight, bias

    # 가중치의 손실기울기를 구하기 위해 필요한 값에 대한 사전작업
    # 전치 행렬
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

## D.2 / E.2 후처리 과정에 대한 순전파 및 역전파 함수 정의

In [16]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output

## D.3 정확도 계산 함수 정의  

In [17]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff

## 단층퍼셉트론 메인 함수 실행

In [18]:
abalone_exec()

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810


## 파라미터 확인

In [19]:
print(weight)
print(bias)

[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]


## 새로운 입력 벡터 X에 대한 예측

In [20]:
x = np.array([0,1,0,0.44,0.3,0.08,0.5,0.23,0.11,0.2])
output = forward_neuralnet(x)
print(output)

(array([8.65881481]), array([0.  , 1.  , 0.  , 0.44, 0.3 , 0.08, 0.5 , 0.23, 0.11, 0.2 ]))


## 하이퍼퍼라미터 수정하며 실험

* LEARNING_RATE = 0.001 -> 0.1
* epoch_count = 10 -> 100
* mb_size = 10 -> 100

In [21]:
LEARNING_RATE = 0.1
abalone_exec(epoch_count=100,mb_size=100,report=20)

Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
